In [1]:
import polars as pl

In [18]:
data1 = pl.read_excel(source='data/data1.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data2 = pl.read_excel(source='data/data2.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data3 = pl.read_excel(source='data/data3.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data4 = pl.read_excel(source='data/data4.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data5 = pl.read_excel(source='data/data5.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")

Оставляем только те строки, где известен победитель и есть ИНН заказчика и поставщика.

In [19]:
fdata1 = data1.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata2 = data2.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata3 = data3.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata4 = data4.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata5 = data5.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("Победитель").is_not_null() & pl.col("ИНН заказчика").is_not_null())

In [146]:
data = pl.concat([fdata1, fdata2, fdata3, fdata5])
data = data.with_columns(pl.col("ИНН поставщика").str.to_integer())
data = pl.concat([data, fdata4])
data.shape

(16240, 18)

In [147]:
data = data.rename({'Дата \nокончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ':'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                    'Дата начала подачи заявок/Дата начала исполнения контракта / Дата публикации ППГ' : 'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                    'Дата  окончания проведения торгов' : 'Дата окончания проведения торгов'})

Оставляем только те строки, где известно снижение на торгах и статус допуска допущен

In [148]:
data = data.filter((pl.col("Статус допуска")==pl.lit("Допущен")) & pl.col("Снижение на торгах,%").is_not_null())
data.shape

(14765, 18)

Уберем нереалистичные значения в ставках в Снижение на торгах,% 

In [151]:
data = data.filter(pl.col('Снижение на торгах,%')>=-100)
data.shape

(14739, 18)

Теперь работаем с датами

In [152]:
ddata = data.with_columns(pl.col('Дата публикации').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания проведения торгов').str.to_date(format="%m-%d-%y", strict=False))

In [156]:
data = ddata.filter(pl.col('Дата публикации').is_not_null() & 
                         pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').is_not_null() & 
                         pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').is_not_null() & 
                         pl.col('Дата окончания проведения торгов').is_not_null())
data.shape

(11460, 18)

In [159]:
data.head(1)

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Сфера деятельности,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,"Снижение на торгах,%",Форма публикации,Тип торгов
i64,str,i64,f64,str,str,str,date,date,date,date,str,i64,str,str,f64,str,str
1,"""МУП ""РЕСУРС""""",6614001840,151972.23,"""[ОКПД2 28.14] Арматура трубопр…","""Свердловская область""","""Свердловская область""",2023-05-30,2023-05-31,2023-05-30,2023-05-30,"""ООО ""САНТЕХКОМПЛЕКТ-ТАГИЛ""""",6623019699,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…"


In [162]:
data.write_excel( dtype_formats={pl.Date: "yyyy-mm-dd"})